<a href="https://colab.research.google.com/github/farroshsy/MidtermExam_BigData/blob/main/FPGrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Midterm Exam**
## Spark Streaming and Reccomendation Systems
---

Name: Farros Hilmi Syafei 
<br>
Student ID: 5025201012
<br>
Class: Big Data A
<br>
Lecturer: Abdul Munif, S.Kom., M.Sc.


## Problem
### 2. Recommendation Systems - Frequent Pattern Mining
You are given the market basket dataset (market-basket.csv). 
*   Use the FP-growth algorithm in Apache Spark to find the most frequent items. Save
your result into an .xlsx files.
*   Do the experiment with different minSupport and minConfidence values. Give the conclusion at the end of experiment.



# Data Driven Dealings Development

- EDA on Sales Data
- RFM Clustering
- Predicting Sales
- Market Basket Analysis
- Recommending Items per Customer

## Market Basket Analysis

In [ ]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [ ]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
# Sets up the environment variables for Java and Spark, initializes the findspark library
# Creates a new SparkSession for a PySpark application named 'fpgrowth'.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46625)
Traceback (most recent call last):
  File "/content/spark-3.0.0-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-3.0.0-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
# To be able to use your data stored in your Google Drive you first need to mount your Google Drive so you can load and save files to it. 
from google.colab import drive
drive.mount('/content/gdrive')
#You'll need to put in a token which Google will generate for you as soon as you click on the link

In [ ]:
import csv
csv_path = '/content/gdrive/MyDrive/BigData/market-basket.csv'
with open(csv_path, 'r') as f:
    temp_lines = f.readline() + '\n' + f.readline()
    # The csv.Sniffer() method is used to detect the delimiter used in the file (either ; or ,) and then the dialect object is used to read the file with the correct delimiter. The pandas.read_csv() method is used to load the data into a pandas DataFrame object called data.
    dialect = csv.Sniffer().sniff(temp_lines, delimiters=';,')
    f.seek(0)
    data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)
data.head()

In [ ]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth

data = data.astype(str)
data['BillNo'] = data['BillNo'].astype(str)

sparkdata = spark.createDataFrame(data)
basketdata = sparkdata.dropDuplicates(['BillNo', 'Itemname']).sort('BillNo')
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')

In [ ]:
data

#### First Experiment
minSupport = 0.002 and minConfidence == 0.002

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.002, minConfidence=0.002) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_Exp1.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_Exp1.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred_Exp1.xlsx')

#### Second Experiment
minSupport = 0.005 and minConfidence == 0.005

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.005, minConfidence=0.005) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_Exp2.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_Exp1.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred_Exp1.xlsx')

#### Third Experiment
minSupport = 0.01 and minConfidence == 0.01

In [ ]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.01, minConfidence=0.01) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfItemsFreq_Exp3.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift_Exp1.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfSalesTransactionIDCollectListPred_Exp1.xlsx')

## Conclusion



*   Minimum Support: This parameter determines the minimum frequency that an itemset must have in order to be considered frequent. Itemsets that occur less frequently than the minimum support threshold are pruned, as they are unlikely to be interesting or informative. Increasing the minimum support value leads to fewer frequent itemsets being generated, while decreasing it leads to more frequent itemsets.
*   Minimum Confidence: This parameter determines the minimum level of confidence that an association rule must have in order to be considered interesting or informative. Association rules that have less confidence than the minimum confidence threshold are considered uninteresting and are pruned. Increasing the minimum confidence value leads to fewer association rules being generated, while decreasing it leads to more association rules.

Sure! In general, the minimum support and minimum confidence parameters in association rule mining algorithms such as FP-growth can have a significant impact on the results generated.

Minimum Support: This parameter determines the minimum frequency that an itemset must have in order to be considered frequent. Itemsets that occur less frequently than the minimum support threshold are pruned, as they are unlikely to be interesting or informative. Increasing the minimum support value leads to fewer frequent itemsets being generated, while decreasing it leads to more frequent itemsets.

Minimum Confidence: This parameter determines the minimum level of confidence that an association rule must have in order to be considered interesting or informative. Association rules that have less confidence than the minimum confidence threshold are considered uninteresting and are pruned. Increasing the minimum confidence value leads to fewer association rules being generated, while decreasing it leads to more association rules.

In general, finding the optimal values for these parameters requires a balance between generating enough frequent itemsets or association rules to be useful and avoiding overfitting or generating too many irrelevant results. This can be achieved through iterative experimentation with different parameter values, as well as domain knowledge and intuition.

